In [1]:
%matplotlib inline 
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import pprint
pp = pprint.PrettyPrinter(indent=4)

import GPy

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
movie = pd.read_csv('../data/movie_easy.csv')
rating = pd.read_csv('../data/rating.csv')
# movie.head()
# rating.head()

In [53]:
#user_id to train on
id=4169

In [4]:
def get_train_for_user(id=1):
    user_ratings = rating[rating['user_id']==id]
    return pd.merge(user_ratings, movie, on='movie_id')

def shuffle(df, f=.8):
    df = df.reindex(np.random.permutation(df.index))
    df.reset_index()
    return df

In [5]:
# get all data for specified user
data = get_train_for_user(id=id).drop(['user_id', 'movie_id', 'title'], axis=1)

In [9]:
# split into features and target
X_data = data.ix[:,1:]
Y_data = data.ix[:,0]
X_data.shape, Y_data.shape

((2264, 7), (2264,))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3, random_state=0)

In [12]:
mm_scaler = preprocessing.MinMaxScaler()
X_train_minmax = mm_scaler.fit_transform(X_train)

In [18]:
y_train.values.shape

(1584,)

In [19]:
X_train1 = X_train_minmax
y_train1 = y_train.values
y_train1 = y_train1.reshape((1584,1))
kernel = GPy.kern.RBF(input_dim=len(X_train_minmax[0]), variance=1., lengthscale=1.)

In [21]:
m = GPy.models.GPRegression(X_train1,y_train1,kernel)
m.optimize()

In [22]:
X_test1 = mm_scaler.transform(X_test)

In [54]:
# predicted values and another output.
# I guess it's the variance but I can't find docs on it
Yhat, conf = m.predict(X_test1)

In [55]:
# probably there's a better way to do this...
act=pd.DataFrame(y_test).reset_index(drop=True)
prd=pd.DataFrame(Yhat)
conf=pd.DataFrame(conf)
diff=pd.concat([act, prd, conf], axis=1,join_axes=[act.index])

In [56]:
diff.columns=['actual', 'predicted', 'conf']

In [47]:
diff['residual'] = diff.apply(lambda r: r['predicted']-r['actual'], axis=1)

In [51]:
diff.sort_values(by='residual').head()

,actual,predicted,conf,residual
163,5,1.973738,0.769943,-3.026262
208,5,2.784773,0.685803,-2.215227
563,5,2.913493,0.692178,-2.086507
659,5,2.926175,0.693752,-2.073825
173,5,2.966545,0.696179,-2.033455


In [52]:
diff.sort_values(by='residual').tail()

,actual,predicted,conf,residual
313,2,4.095503,0.686103,2.095503
575,1,3.255775,0.675853,2.255775
119,1,3.379527,0.685065,2.379527
501,1,4.354160,0.688432,3.354160
389,1,4.357669,0.684140,3.357669
